In [ ]:
import sys
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
from functools import partial

np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

# Lab 1: Tackling Fashion MNIST With a CNN

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_valid = (X_valid - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

# Lab 2: Using a Pretrained Model

In [ ]:
import numpy as np
from sklearn.datasets import load_sample_image

# Load sample images
china = load_sample_image("china.jpg") / 255
flower = load_sample_image("flower.jpg") / 255
images = np.array([china, flower])
batch_size, height, width, channels = images.shape

In [ ]:
china_box = [0, 0.03, 1, 0.68]
flower_box = [0.19, 0.26, 0.86, 0.7]
images_resized = tf.image.crop_and_resize(images, [china_box, flower_box], [0, 1], [224, 224])
plot_color_image(images_resized[0])
plt.show()
plot_color_image(images_resized[1])
plt.show()

In [ ]:
top_K = keras.applications.resnet50.decode_predictions(Y_proba, top=3)
for image_index in range(len(images)):
    print("Image #{}".format(image_index))
    for class_id, name, y_proba in top_K[image_index]:
        print("  {} - {:12s} {:.2f}%".format(class_id, name, y_proba * 100))
    print()

# Lab 3: Pretrained Models for Transfer Learning

In [ ]:
plt.figure(figsize=(12, 10))
index = 0
for image, label in train_set_raw.take(9):
    index += 1
    plt.subplot(3, 3, index)
    plt.imshow(image)
    plt.title("Class: {}".format(class_names[label]))
    plt.axis("off")

plt.show()

In [ ]:
def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop
    return image[top_crop:bottom_crop, left_crop:right_crop]

def random_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
    return tf.image.random_crop(image, [min_dim, min_dim, 3])

def preprocess(image, label, randomize=False):
    if randomize:
        cropped_image = random_crop(image)
        cropped_image = tf.image.random_flip_left_right(cropped_image)
    else:
        cropped_image = central_crop(image)
    resized_image = tf.image.resize(cropped_image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

batch_size = 32
train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(partial(preprocess, randomize=True)).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

# 연습문제(Exercise)

## Question 1
Build a CNN model to classify fashion mnist dataset.

* np.random.seed(42)와 tf.random.set_seed(42) 반드시 42로 설정해야합니다.

**Model architecture**
 * Input layer
 * Conv(filters=64,kernel_size=3)
 * Conv(filters=64,kernel_size=3)
 * MaxPooling(pool_size=2)
 * Conv(filters=128,kernel_size=3)
 * Conv(filters=128,kernel_size=3)
 * MaxPooling(pool_size=2)
 * Conv(filters=256,kernel_size=3)
 * Conv(filters=256,kernel_size=3)
 * Conv(filters=256,kernel_size=3)
 * MaxPooling(pool_size=2)
 * Flatten()
 * Dense(128)
 * Output layer

Train the model with a proper optimizer and learning rate for 10 epochs.  Report accuracy on test set.


정답은 Test의 정확도가 0.9보다 크거나 같아야합니다.


### 1. 데이터 로드
### 1.1 패션 MNIST 데이터 로드하시오.

### 1.2 학습 데이터로 검증(vaild) 데이터 만들시오.
* 인덱스 번호 기준 -50000번 입니다.

### 1.3 표준화(Standard Scale) 시키시오.

### 1.4 이미지 shape을 (28,28) -> (28,28,1)로 변환하시오.

### 2. 모델 빌드
### 2.1 모델 구축하시오.

### 2.2 모델 loss 및 opimizer 설정하시오.
* metrics 는 정확도로 설정하시오.

### 2.3 학습

### 3. 성능 평가하시오.
* 정답은 Test의 정확도가 0.9보다 크거나 같아야합니다.

## Question 2
 For the task in lab 3, use InceptionV3 (pretrained on ImageNet) as base model and train the model again. Report accuracy on test set.
* np.random.seed(42)와 tf.random.set_seed(42) 반드시 42로 설정해야한다.


정답은 Test의 정확도가 0.9보다 크거나 같아야합니다.